### 깃허브에서 가져온 데이터 확인

In [ ]:
import pandas as pd


file_name = "github_dataset.csv"


print(f"원본 파일 '{file_name}'을 불러옵니다...")


try:

    df = pd.read_csv(file_name, engine="python")


    print("\n✅ 원본 파일 로드 성공!")

    print("데이터프레임 정보:")

    df.info()


    print(f"\n전체 행 개수: {len(df)}")

    print("\n데이터 미리보기:")

    print(df.tail())


except FileNotFoundError:
    print(

        f"오류: '{file_name}' 파일을 찾을 수 없습니다. 파일이 코드와 같은 폴더에 있는지 확인해주세요."
    )

except Exception as e:

    print(f"파일을 읽는 중 오류가 발생했습니다: {e}")

원본 파일 'github_dataset.csv'을 불러옵니다...

✅ 원본 파일 로드 성공!
데이터프레임 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2927 entries, 0 to 2926
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          2927 non-null   int64  
 1   transcript  2927 non-null   object 
 2   confidence  0 non-null      float64
 3   label       2927 non-null   int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 91.6+ KB

전체 행 개수: 2927

데이터 미리보기:
       id                                         transcript  confidence  \
2922  690  지금까지 사실확인서를 연락드렸습니다. 하고 있는데 이런 사건 때문에 내가 여쭤볼게 ...         NaN   
2923  691  연료 데이는 사건 때문에 연락 드렸는데 혹시 김명철씨 가십니까 모르시고 아는 사람도...         NaN   
2924  692  4장 4절 일어났습니까? 장소 잡고 님께서는 42세 남성 김동술 알고 계십니까? 아...         NaN   
2925  693  네\n네 여보세요\n뭐 어디세요?\n아 네 안녕하세요 저희 그 하나캐피탈에서 연락을...         NaN   
2926  694  통장명이 (삐-) 계좌 맞구요? 그리고 통장 발급 일은 2017년 3월 14일입니다...         NaN   

      label  
2922      1  
2923      

In [67]:
df.iloc[2231]

id                                                         2231
transcript    저는 학교를 졸업한 지 오래되어서 어~ 기억이 쪼금 먼 기리 멀기는 하지만 어~ 저...
confidence                                                  NaN
label                                                         0
Name: 2231, dtype: object

### 보이스피싱 대화, 일반 대화 행 인덱스 기준으로 데이터 프레임 분리 후 컬럼명 수정

In [ ]:
import os



FILE_TO_READ = "github_dataset.csv"


PHISHING_ROW_INDICES = list(range(2232, 2927))



try:


    current_dir = os.getcwd()


    file_path = os.path.join(current_dir, FILE_TO_READ)



    print(f"파일을 다음 경로에서 찾습니다: {file_path}")



    df = pd.read_csv(file_path, engine='python')


    print(f"\n✅ '{FILE_TO_READ}' 파일을 성공적으로 불러왔습니다. (총 {len(df)}개 행)")



    phishing_df = df.iloc[PHISHING_ROW_INDICES].copy()


    normal_df = df[~df.index.isin(PHISHING_ROW_INDICES)].copy()



    phishing_final_df = phishing_df.rename(columns={


        'id': 'file_name',


        'confidence': 'phishing_type',


        'label': 'speaker',


        'transcript': 'text'


    })



    phishing_final_df = phishing_final_df[['file_name', 'phishing_type', 'speaker', 'text']]




    normal_final_df = normal_df.rename(columns={


        'id': 'file_name',


        'transcript': 'text'


    })



    normal_final_df = normal_final_df[['file_name', 'text']]



    phishing_final_df.to_csv("phishing_data.csv", index=False, encoding="utf-8-sig")


    normal_final_df.to_csv("normal_data.csv", index=False, encoding="utf-8-sig")



    print("\n 작업이 완료되었습니다.")
    print(


        f"피싱 데이터가 'phishing_data.csv' 파일로 저장되었습니다. (총 {len(phishing_final_df)}개 행)"
    )
    print(


        f"일반 데이터가 'normal_data.csv' 파일로 저장되었습니다. (총 {len(normal_final_df)}개 행)"
    )



except FileNotFoundError:


    print(f"\n 오류: '{file_path}' 경로에 파일이 없습니다.")


except IndexError:
    print(


        f"\n 오류: 'PHISHING_ROW_INDICES'에 지정된 행 번호가 실제 파일의 전체 행 수를 벗어났습니다."
    )


except KeyError as e:


    print(f"\n 오류: '{e}' 컬럼을 찾을 수 없습니다. 원본 파일에 해당 컬럼이 있는지 확인해주세요.")


except Exception as e:


    print(f"오류가 발생했습니다: {e}")

파일을 다음 경로에서 찾습니다: c:\Users\user\Downloads\woogawooga\woogawooga_project\dataset_create\github_dataset.csv

✅ 'github_dataset.csv' 파일을 성공적으로 불러왔습니다. (총 2927개 행)

 작업이 완료되었습니다.
피싱 데이터가 'phishing_data.csv' 파일로 저장되었습니다. (총 695개 행)
일반 데이터가 'normal_data.csv' 파일로 저장되었습니다. (총 2232개 행)


### 문단 일반대화 데이터 (.,?!)문장단위로 분리

In [ ]:
import re


try:

    df = pd.read_csv("normal_data.csv")

except FileNotFoundError:

    print("오류: 'normal_data.csv' 파일을 찾을 수 없습니다.")

    df = pd.DataFrame()


if not df.empty:

    df["sentences"] = df["text"].apply(

        lambda x: [

            s.strip() + d

            for s, d in zip(

                re.split(r"([.?!])", str(x))[:-1:2], re.split(r"([.?!])", str(x))[1::2]
            )

        ]
    )


    df_exploded = df.explode("sentences")

    df_final = df_exploded.drop(columns=["text"]).rename(columns={"sentences": "text"})

    df_final = df_final[["file_name", "text"]]

    df_final = df_final.dropna(subset=["text"])

    df_final = df_final[df_final["text"].str.strip() != ""]


    print("✅ 작업 완료! 문장 분리 결과 (상위 10개):")

    print(df_final.head(10))


    df_final.to_csv('sentences_data.csv', index=False, encoding='utf-8-sig')

    print("\n'sentences_data.csv' 파일로 저장되었습니다.")

✅ 작업 완료! 문장 분리 결과 (상위 10개):
   file_name                                               text
0          0                           저는 여행 다니는 것을 굉장히 좋아하는데요.
0          0  그래가지고 스페인이나 뭐 영국 유럽 아니면 국내에서도 뭐 강릉이나 전주 같은 데를 ...
0          0        저 여행 다니는 거 되게 좋아해서 대학교 내내 여행을 엄청 많이 다녔었는데요.
0          0  제가 고등학교 때는 여행에 대해 흥미가 없었는데 그게 좀 아버지가 짠대로 패키지처럼...
0          0  그래서 대학교 간 이후로는 해외여행을 되게 많이 갔었는데 그중에서 제일 기 좋았던 ...
0          0                어~ 혹시 포르투갈이나 스페인 유럽 쪽 다녀오신 적 있으신가요?
0          0                                              어~ 네.
0          0                        저도 우연히 스페인과 포르투갈을 다녀왔었었습니다.
0          0  어~ 저는 스페인 중에서도 마드리드에 근교에 있었던 톨레도라는 지역이 굉장히 좋았는데요.
0          0  그 톨레도에서 특히 기억에 남았던 거는 거기에 대성당이 있는데 그 성당이 엄청 화려...

'sentences_data.csv' 파일로 저장되었습니다.
